**Convolutional Neural Network (CNN) basierend auf die YOLO-Architektur zur Erkennung von deutschen Verkehrszeichen in Echtzeit mit dem GTSRB-Dataset**

Autoren: Laieb Ahmad, Enes Bayar, Mehmet Akif Onur & Mert-Can Öksuz
<br>
Datum: 19.08.2024
<br>
Modul: KI-DL/ML

## **Befehle zum Installieren/Upgraden von Packages** 
Falls Bedarf da ist (Inkompatibilität der Packages, wie oben beschrieben), können diese Befehle direkt im Kernel ausgeführt werden, um die entsprechenden Packages upzugraden.

In [ ]:
%pip install numpy --upgrade
%pip install pandas --upgrade
%pip install scikit-learn --upgrade
%pip install scikit-image --upgrade
%pip install pillow --upgrade
%pip install imbalanced-learn --upgrade
%pip install statsmodels --upgrade
%pip install matplotlib --upgrade
%pip install seaborn --upgrade
%pip install tensorflow --upgrade
%pip install ultralytics --upgrade

## **Laden von Packages** 
Hier Importieren wir die benötigten Packages/Frameworks, um unser CNN-Modell aufzubauen, anzutrainieren und in den Use-Case überzuleiten.

In [9]:
from ultralytics import YOLO
import os
import numpy as np
import pandas as pd
import cv2
import shutil
import matplotlib
import csv
import torch

## **Übersicht der Versionen von den genutzten Packages** 
Eine Übersicht über die Versionen der einzelnen Packages/Frameworks. Das dient dazu, die Kompitabilität der Packages mit der Python-Version (oder mit der miniconda-Version) zu überwachen.

In [ ]:
print("Package 'Numpy':",np.__version__)
print("Package 'Pandas':",pd.__version__)
print("Package 'OpenCV':",cv2.__version__)
print("Package 'Matplotlib':",matplotlib.__version__)
print("Package 'CSV':",csv.__version__)
print("Package 'PyTorch':",torch.__version__)